# Лабораторная работа 2. Вариант 1.

In [1]:
import numpy as np
from numpy import linalg as LA
from IPython.display import display, Math, Latex


def find_max_eigenvalue(A):
    return np.amax(LA.eigvals(A))


def find_inverse(A):
    shape = A.shape
    if len(shape) != 2 or shape[0] != shape[1]:
        raise ValueError('A is not square 2d matrix')

    n = len(A)

    B = np.eye(n)

    # Elimination
    for i in range(n):
        if A[i][i] == 0:
            raise ValueError('Pivot is zero applying Gauss Jordan elimination')

        for j in range(n):
            if i != j:
                ratio = A[j, i] / A[i, i]

                for k in range(n):
                    A[j][k] = A[j][k] - ratio * A[i][k]
                    B[j][k] = B[j][k] - ratio * B[i][k]

    # Normalization
    for i in range(n):
        B[i] /= A[i][i]

    return B



# P.94 V.1
A = np.array([[3.278164, 1.046583, -2.06459],
              [1.046583,  2.975937,  0.934251],
              [-1.378574, 0.934251, 4.836173]])

b = np.array([[-0.527466],
              [2.526877],
              [5.165441]])

accuracy = 9

## Дана линейная система Ax=b

In [2]:
print('A =\n', A)
print('b =\n', b)

A =
 [[ 3.278164  1.046583 -2.06459 ]
 [ 1.046583  2.975937  0.934251]
 [-1.378574  0.934251  4.836173]]
b =
 [[-0.527466]
 [ 2.526877]
 [ 5.165441]]


### 1) Найти решение $x^{∗}$ методом Гаусса.

In [3]:
def find_solution_gauss(A, b, use_pivoting=False):
    n = len(A)
    if b.size != n:
        raise ValueError('Incompatible sizes of A and b')

    for i in range(n - 1):
        if use_pivoting:
            pivot_index = abs(A[i:, i]).argmax() + i
            if A[pivot_index, i] == 0:
                raise ValueError('A is singular')

            if pivot_index != i:
                A[[i, pivot_index]] = A[[pivot_index, i]]
                b[[i, pivot_index]] = b[[pivot_index, i]]
        else:
            if A[i, i] == 0:
                raise ValueError('Pivot is zero')

        for j in range(i + 1, n):
            ratio = A[j, i] / A[i, i]
            A[j, i:] -= ratio * A[i, i:]
            b[j] -= ratio * b[i]

    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i + 1:], x[i + 1:])) / A[i, i]

    return x

x_gauss = find_solution_gauss(np.copy(A), np.copy(b))
x_sln = np.array([x_gauss]).T
print(x_sln)

[[0.44116647]
 [0.33976985]
 [1.12820433]]


### 2) Преобразовать исходную систему к системе вида $x = H_Dx+g_D$ , где $H_D = E-D^{-1}A, g_D = D^{-1}b$. Здесь D — диагональная матрица, у которой на диагонали находятся диагональные элементы матрицы A. Вычислить $\|H_D\|_\infty$.

In [4]:
D = np.diag(np.diag(np.copy(A)))
print('D =\n', D)
Hd = np.eye(len(A)) - LA.inv(np.copy(D)).dot(A).round(accuracy)
print('\nHd =\n', Hd)
gd = LA.inv(np.copy(D)).dot(b).round(accuracy)
print('\ngd =\n', gd)
hd_norm = LA.norm(Hd, np.inf)
print('\n||Hd|| =\n', hd_norm)

D =
 [[3.278164 0.       0.      ]
 [0.       2.975937 0.      ]
 [0.       0.       4.836173]]

Hd =
 [[ 0.         -0.31925889  0.62980071]
 [-0.35168184  0.         -0.31393507]
 [ 0.28505473 -0.19317981  0.        ]]

gd =
 [[-0.16090287]
 [ 0.84910299]
 [ 1.06808441]]

||Hd|| =
 0.9490595960000001


### 3) Вычислить априорную оценку погрешности $\|x^{(7)}-x^*\|_\infty$.

In [5]:
def count_aprior_error(H, g, x0, k):
    return LA.norm(H, np.inf) ** k * LA.norm(x0, np.inf) + (LA.norm(H, np.inf) ** k / (1 - LA.norm(H, np.inf))) * LA.norm(g, np.inf)
    
aprior_error = count_aprior_error(Hd, gd, np.zeros((len(Hd), 1)), 7)
print('Априорная оценка: ||x_7−x*|| <= ', aprior_error)

Априорная оценка: ||x_7−x*|| <=  14.541110854852617


### 4) Вычислить приближение $x^{(7)}$ методом простой итерации. Вывести его фактическую погрешность, апостериорную оценку, априорную оценку. Уточнить последнее приближение по Люстернику. Вывести его фактическую погрешность.

In [6]:
def find_solution_simple_iteration(H, g, k):
    if k < 0:
        raise ValueError('k can not be negative')
    if k == 0:
        return np.zeros((len(H), 1))
    return H.dot(find_solution_simple_iteration(H, g, k - 1)) + g


def find_solution_lusternik(H, g, k):
    xk = find_solution_simple_iteration(H, g, k)
    xk1 = find_solution_simple_iteration(H, g, k - 1)
    p = find_max_eigenvalue(H)
    return xk1 + (1 / (1 - p)) * (xk - xk1)


def count_aposterior_error(H, xk, xk1):
    return (LA.norm(H, np.inf) / (1 - LA.norm(H, np.inf))) * LA.norm(xk - xk1, np.inf)

x_6 = find_solution_simple_iteration(Hd, gd, 6)
x_7 = find_solution_simple_iteration(Hd, gd, 7)
aposterior_error = count_aposterior_error(Hd, x_7, x_6)
print('x_7 =\n', x_7)
print('Метод простой итерации, апостериорная оценка: ||x_7 - x*|| <= ', aposterior_error)
print('Метод простой итерации, априорная оценка: ||x_7 - x*|| <= ', aprior_error)
simple_iteration_error = LA.norm(x_7 - x_sln, np.inf)
print('Метод простой итерации: ||x_7 - x*|| = ', simple_iteration_error)
x_lusternik = find_solution_lusternik(Hd, gd, 7)
print('Приближение по Люстернику: x_7 =\n', x_lusternik)
lusternik_error = LA.norm(x_lusternik - x_sln, np.inf)
print('Приближение по Люстернику: ||x_7 - x*|| = ', lusternik_error)

x_7 =
 [[0.39774661]
 [0.37325093]
 [1.10168369]]
Метод простой итерации, апостериорная оценка: ||x_7 - x*|| <=  0.3550125173127319
Метод простой итерации, априорная оценка: ||x_7 - x*|| <=  14.541110854852617
Метод простой итерации: ||x_7 - x*|| =  0.04341985944526122
Приближение по Люстернику: x_7 =
 [[0.42707658]
 [0.33336927]
 [1.13473128]]
Приближение по Люстернику: ||x_7 - x*|| =  0.014089880238567887


### 5) Вычислить приближение $x^{(7)}$ к решению системы $x = H_Dx + g_D$ методом Зейделя. Вывести его фактическую погрешность. Сравнить с решением, полученным методом простой итерации.

In [11]:
def find_solution_seidel(Hl, Hr, g, k):
    if k < 0:
        raise ValueError('k can not be negative')
    if k == 0:
        return np.zeros((len(Hl), 1))
    
    x_prev = find_solution_seidel(Hl, Hr, g, k - 1)
    E = np.eye(len(Hl))
    
    return LA.inv(E - Hl).dot(Hr).dot(x_prev) + LA.inv(E - Hl).dot(g)

Hdl = np.copy(Hd)
Hdr = np.copy(Hd)

for i in range(len(Hd)):
    for j in range(len(Hd)):
        if i > j:
            Hdr[i, j] = 0
        else:
            Hdl[i, j] = 0
            
x_seidel = find_solution_seidel(Hdl, Hdr, gd, 7)
print('Приближение методом Зейделя:\n', x_seidel)
seidel_error = LA.norm(x_seidel - x_sln, np.inf)
print('Приближение методом Зейделя: ||x_7 - x*|| = ', seidel_error)

Приближение методом Зейделя:
 [[0.43418382]
 [0.34417454]
 [1.12536299]]
Приближение методом Зейделя: ||x_7 - x*|| =  0.0069826427664058666


### 6) При выполнении задания в математическом пакете определить спектральный радиус матрицы перехода, если рассматривать метод Зейделя как метод простой итерации.

### 7) Вычислить приближение $x^{(7)}$ методом верхней релаксации.

In [8]:
def find_solution_upper_relaxation(H, g, k):
    return 0

upper_relaxation_error = LA.norm(0 - x_sln, np.inf)

### Сравнить фактические погрешности $x^{(7)}$, полученного различными методами.

In [12]:
print('Фактические погрешности для x_7')
print('Метод простой итерации: ', simple_iteration_error)
print('Приближение по Люстернику: ', lusternik_error)
print('Метод Зейделя: ', seidel_error)
print('Метод верхней релаксации: ', upper_relaxation_error)

Фактические погрешности для x_7
Метод простой итерации:  0.04341985944526122
Приближение по Люстернику:  0.014089880238567887
Метод Зейделя:  0.0069826427664058666
Метод верхней релаксации:  1.1282043259561196
